In [1]:
# Setup imports
import os
import sys

# Fix paths
sys.path.append('../')

# Run django init
import django
django.setup()

from django.db import connection

In [2]:
# Packages
import pandas as pd
import numpy as np

# Model imports
from ossld.models import (PyPiClassification, PyPiPackage, PyPiPackageClassification, PyPiPackageInfo,
                          PyPiPackageRelease, PyPiPackageReleaseImport, License, )

## Table 1: Total counts for all primary data model objects reported in paper

In [3]:
package_count = PyPiPackage.objects.count()
print("Number of packages:", package_count)

release_count = PyPiPackageRelease.objects.count()
print("Number of releases:", release_count)

classification_count = PyPiPackageClassification.objects.count()
print("Number of classifications assigned:", classification_count)

author_count = PyPiPackageInfo.objects.values('author').distinct().count()
print("Number of authors:", author_count)

maintainer_count = PyPiPackageInfo.objects.values('maintainer').distinct().count()
print("Number of maintainers:", maintainer_count)

license_raw_count = PyPiPackageInfo.objects.values('license').distinct().count()
print("Number of licenses (raw):", license_raw_count)

import_count = PyPiPackageReleaseImport.objects.count()
print("Number of imports:", import_count)

Number of packages: 178952
Number of releases: 1745744
Number of classifications assigned: 947896
Number of authors: 76997
Number of maintainers: 3047
Number of licenses (raw): 4610
Number of imports: 156816750


### Date Range Coverage

In [4]:
# Date ranges
# FIRST_RELEASE_DATE
first_release = PyPiPackageRelease.objects.earliest('upload_time').upload_time
print("First release:", first_release)

# LAST_RELEASE_DATE
last_release = PyPiPackageRelease.objects.latest('upload_time').upload_time
print("Last release:", last_release)

print("Duration", last_release-first_release)

First release: 2005-03-21 15:59:25+00:00
Last release: 2019-05-15 20:29:29+00:00
Duration 5168 days, 4:30:04


## Table 2: Primary counts by year

In [11]:
# New packages
first_release_by_package = pd.read_sql_query("""
    SELECT pp.name AS package_name, EXTRACT(year FROM ppi.first_release_date) AS first_release_year
    FROM ossld_pypipackageinfo ppi
    JOIN ossld_pypipackage pp ON pp.id = ppi.pypi_package_id
    """, connection)

first_release_by_year = first_release_by_package["first_release_year"].value_counts().sort_index()
first_release_by_year.index = [int(year) for year in first_release_by_year.index]

# Active packages
raw_package_release_year = pd.read_sql_query("""
    SELECT EXTRACT(year FROM ppr.upload_time) AS release_year, pypi_package_id AS package_id
    FROM ossld_pypipackagerelease ppr
    JOIN ossld_pypipackage pp ON pp.id = ppr.pypi_package_id
    """, connection)


active_package_by_year = raw_package_release_year.groupby("release_year")["package_id"].nunique()
active_package_by_year.index = [int(year) for year in active_package_by_year.index]

# New releases
release_by_year_df = pd.read_sql_query("""
    SELECT EXTRACT(YEAR FROM ppr.upload_time) AS year, COUNT(*) AS count
    FROM ossld_pypipackagerelease ppr
    GROUP BY year
    ORDER BY year ASC
    """, connection, index_col="year")

release_by_year_df.index = [int(year) for year in release_by_year_df.index]

# New authors
author_release_df = pd.read_sql_query("""
    SELECT ppi.author, ppi.author_email, EXTRACT(YEAR FROM ppi.first_release_date) AS first_release_year
    FROM ossld_pypipackageinfo ppi
    """, connection)

author_first_release_df = author_release_df.groupby("author")["first_release_year"].min()
author_email_first_release_df = author_release_df.groupby("author_email")["first_release_year"].min()
author_email_first_release_df = author_email_first_release_df.value_counts().sort_index()
author_email_first_release_df.index = [int(year) for year in author_email_first_release_df.index]

# Merge all
merge_ts_df = pd.concat([first_release_by_year,
                         active_package_by_year,
                         release_by_year_df,
                         author_email_first_release_df
                        ], axis=1)
merge_ts_df.columns = ["new_packages", "active_packages", "new_releases", "new_authors"]
print(merge_ts_df.to_latex())

\begin{tabular}{lrrrr}
\toprule
{} &  new\_packages &  active\_packages &  new\_releases &  new\_authors \\
\midrule
2005 &            96 &               96 &           389 &           68 \\
2006 &           367 &              420 &          2324 &          216 \\
2007 &           876 &             1047 &          5301 &          341 \\
2008 &          1702 &             2223 &         10923 &          637 \\
2009 &          2559 &             3727 &         17426 &          974 \\
2010 &          3522 &             5454 &         23474 &         1455 \\
2011 &          4840 &             7479 &         31107 &         1996 \\
2012 &          7235 &            11047 &         47377 &         3062 \\
2013 &         10438 &            16267 &         74003 &         4633 \\
2014 &         13352 &            21555 &        112994 &         6016 \\
2015 &         17355 &            28498 &        173437 &         7742 \\
2016 &         21849 &            36557 &        253262 &        1029

## Table 3: compound annual growth rates

In [23]:
start_year = 2006
end_year = 2018

# calculate total packages
merge_ts_df.loc[:, "total_packages"] = merge_ts_df["new_packages"].expanding().sum()

new_package_cagr = np.exp((np.log(merge_ts_df.loc[end_year, "new_packages"]) - np.log(merge_ts_df.loc[start_year, "new_packages"])) / (end_year - start_year + 1)) - 1
active_package_cagr = np.exp((np.log(merge_ts_df.loc[end_year, "active_packages"]) - np.log(merge_ts_df.loc[start_year, "active_packages"])) / (end_year - start_year + 1)) - 1
new_release_cagr = np.exp((np.log(merge_ts_df.loc[end_year, "new_releases"]) - np.log(merge_ts_df.loc[start_year, "new_releases"])) / (end_year - start_year + 1)) - 1
new_author_cagr = np.exp((np.log(merge_ts_df.loc[end_year, "new_authors"]) - np.log(merge_ts_df.loc[start_year, "new_authors"])) / (end_year - start_year + 1)) - 1

In [32]:
print(pd.Series({"New Packages": np.round(100. * new_package_cagr, 2),
                "Active Packages": np.round(100. * active_package_cagr, 2),
                 "New Releases": np.round(100. * new_release_cagr, 2),
                 "New Authors": np.round(100. * new_author_cagr, 2)                
                }).to_latex())

\begin{tabular}{lr}
\toprule
{} &      0 \\
\midrule
New Packages    &  43.28 \\
Active Packages &  47.31 \\
New Releases    &  51.21 \\
New Authors     &  39.30 \\
\bottomrule
\end{tabular}



## Table 4: Number of releases per package and inter-release timing

In [59]:
package_release_count_df = pd.read_sql_query("""
    SELECT pp.name, ppi.first_release_date, ppi.last_release_date, ppi.release_count 
    FROM ossld_pypipackageinfo ppi
    JOIN ossld_pypipackage pp ON pp.id = ppi.pypi_package_id
    """, connection)

release_count_summary = package_release_count_df["release_count"].describe()

In [62]:
# Calculate inter-release timing distribution data
package_release_count_df.loc[:, "package_lifespan"] = (package_release_count_df.loc[:, "last_release_date"] - \
    package_release_count_df.loc[:, "first_release_date"]).apply(lambda x: x.days)
package_release_count_df.loc[:, "interrelease_timing"] = None
package_release_count_df.loc[package_release_count_df.loc[:, "release_count"] > 1, "interrelease_timing"] = package_release_count_df.loc[:, "package_lifespan"] / \
    (package_release_count_df.loc[package_release_count_df.loc[:, "release_count"] > 1, "release_count"] - 1)

In [63]:
interrelease_timing_summary = package_release_count_df.loc[:, "interrelease_timing"].astype(float).describe()

In [68]:
print(pd.concat([release_count_summary, interrelease_timing_summary], axis=1).round(2).iloc[1:].to_latex())

\begin{tabular}{lrr}
\toprule
{} &  release\_count &  interrelease\_timing \\
\midrule
mean &           6.93 &                65.38 \\
std  &          19.46 &               144.89 \\
min  &           1.00 &                 0.00 \\
25\%  &           1.00 &                 1.18 \\
50\%  &           3.00 &                16.37 \\
75\%  &           7.00 &                68.12 \\
max  &        4333.00 &              4369.00 \\
\bottomrule
\end{tabular}



## Table 5: Number of packages and releases per author

In [ ]:
author_package_df = pd.read_sql_query("""
    SELECT pp.name, ppi.author_email, ppi.release_count 
    FROM ossld_pypipackageinfo ppi
    JOIN ossld_pypipackage pp ON pp.id = ppi.pypi_package_id
    """, connection)
author_package_df = author_package_df.loc[-(author_package_df["author_email"].isin(["UNKNOWN", "TODO"])) &
                                           (author_package_df["author_email"].apply(lambda x: len(x.strip()) > 0 if x is not None else False)), :]
author_package_count_df = author_package_df.groupby("author_email")["name"].nunique()
print(author_package_count_df.sort_values(ascending=False).describe().to_latex())


In [71]:
author_raw_df = pd.read_sql_query("""
    SELECT ppi.author, COUNT(ppi.pypi_package_id)
    FROM ossld_pypipackageinfo ppi
    GROUP BY ppi.author
    """, connection)

In [72]:
final_token_list = ['inc', 'ltd', 'lda', 'sl', 'sa', 'srl', 'bv', 'corp', 'sro', 'plc', 'llc', 'gmbh', 'ag']
sub_token_list = ["Team", "Organization", "Lab", "Association", "Foundation", "Developers", "Corporation",  "Industr", "Group"]

def is_string_org(input_string):
    if input_string is None or len(input_string.strip()) == 0:
        return False
    
    for sub_token in sub_token_list:
        if " " + sub_token.lower() in input_string.lower():
            return True
    
    final_token = input_string.split().pop().replace(".", "").strip().lower()
    if final_token in final_token_list:
        return True
    
    return False

In [73]:
author_raw_df.loc[:, "is_org"] = author_raw_df["author"].apply(is_string_org)
author_raw_df.loc[:, "is_multiple"] = author_raw_df["author"].str.contains(",") | author_raw_df["author"].str.contains(" and ") | author_raw_df["author"].str.contains("et al.")

## Table 6: Proportion of packages and authors likely to be organization or multiple

In [81]:
org_pct_df = author_raw_df.groupby("is_org")["count"].sum() / author_raw_df["count"].sum()
multiple_pct_df = author_raw_df.groupby("is_multiple")["count"].sum() / author_raw_df["count"].sum()

print(pd.DataFrame({"Author String": 
                  {"Organization": author_raw_df["is_org"].mean(),
                   "Multiple": author_raw_df["is_multiple"].mean()},
              "Packages": 
                  {"Organization": org_pct_df.iloc[1],
                   "Multiple": multiple_pct_df.iloc[1]}
             }).to_latex())

\begin{tabular}{lrr}
\toprule
{} &  Author String &  Packages \\
\midrule
Multiple     &       0.083016 &  0.070052 \\
Organization &       0.054911 &  0.077691 \\
\bottomrule
\end{tabular}



## Table 7: Distribution of package and release sizes

In [84]:
# get release size data
release_df = pd.read_sql_query("""
    SELECT pp.name AS package_name, ppr.upload_time, ppr.package_type, ppr.python_version, ppr.size, ppr.filename
    FROM ossld_pypipackagerelease ppr
    JOIN ossld_pypipackage pp ON pp.id = ppr.pypi_package_id
    """, connection, index_col="package_name")


package_size_df = (release_df.groupby("package_name")["size"].sum() / 1024.).describe()
release_size_df = (release_df["size"] / 1024).describe()

print(pd.concat([package_size_df, release_size_df], axis=1).round(2).iloc[1:].to_latex())

\begin{tabular}{lrr}
\toprule
{} &          size &       size \\
\midrule
mean &  1.646679e+04 &    1606.55 \\
std  &  8.091365e+05 &   14971.49 \\
min  &  2.000000e-02 &       0.00 \\
25\%  &  1.004000e+01 &       7.50 \\
50\%  &  4.000000e+01 &      22.60 \\
75\%  &  2.162800e+02 &     128.16 \\
max  &  1.757637e+08 &  592437.94 \\
\bottomrule
\end{tabular}



## Table 8: Number of packages by license

In [85]:
package_license_df = pd.read_csv("./package_license_final.csv", encoding="utf-8", low_memory=False)

In [86]:
license_family_count = package_license_df.loc[:, "license_final_family"].value_counts()
license_family_count_df = pd.DataFrame(license_family_count)
license_family_count_df.columns = ["count"]
license_family_count_df.loc[:, "percent"] = license_family_count_df["count"] / license_family_count_df["count"].sum() 
print(license_family_count_df.to_latex())

\begin{tabular}{lrr}
\toprule
{} &  count &   percent \\
\midrule
MIT           &  60945 &  0.340566 \\
Unknown       &  48742 &  0.272375 \\
GPL           &  29403 &  0.164307 \\
BSD           &  20094 &  0.112287 \\
Apache        &  15004 &  0.083844 \\
Public Domain &   1194 &  0.006672 \\
Zope          &   1150 &  0.006426 \\
ISC           &    719 &  0.004018 \\
MPL           &    712 &  0.003979 \\
PSFL          &    524 &  0.002928 \\
Proprietary   &    190 &  0.001062 \\
CC            &    178 &  0.000995 \\
CeCILL        &     72 &  0.000402 \\
zlib          &     25 &  0.000140 \\
\bottomrule
\end{tabular}



## Table 9: Number of packages by GPL license

In [88]:
license_gpl_family_version_count = package_license_df.loc[package_license_df["license_final_family"]=="GPL", :]\
    .groupby(["license_final_normalized", "license_final_version"])["package_id"].count()
license_gpl_family_version_count_df = pd.DataFrame(license_gpl_family_version_count)
license_gpl_family_version_count_df.columns = ["count"]
license_gpl_family_version_count_df.loc[:, "percent"] = license_gpl_family_version_count_df["count"] / license_gpl_family_version_count_df["count"].sum() 
print(license_gpl_family_version_count_df.to_latex())

\begin{tabular}{llrr}
\toprule
     &         &  count &   percent \\
license\_final\_normalized & license\_final\_version &        &           \\
\midrule
AGPL & 3 &   5642 &  0.191885 \\
     & Unknown &    683 &  0.023229 \\
GPL & 2 &   2428 &  0.082577 \\
     & 2.1 &     69 &  0.002347 \\
     & 3 &   8437 &  0.286944 \\
     & Unknown &   8508 &  0.289358 \\
LGPL & 2 &    122 &  0.004149 \\
     & 2.1 &     90 &  0.003061 \\
     & 3 &   1501 &  0.051049 \\
     & Unknown &   1923 &  0.065401 \\
\bottomrule
\end{tabular}



## Table 10: Development Status

In [89]:
development_status_df = pd.read_sql_query("""
    SELECT pc.classification AS classification, COUNT(*) AS count
    FROM ossld_pypipackageclassification ppc
    JOIN ossld_pypiclassification pc ON ppc.pypi_classification_id = pc.id
    WHERE pc.classification LIKE 'Devel%'
    GROUP BY classification
    ORDER BY count DESC
    """, connection, index_col="classification")
development_status_df = development_status_df.sort_index()
development_status_df.loc[:, "percent"] = development_status_df["count"] / development_status_df["count"].sum()
print(development_status_df.to_latex())

\begin{tabular}{lrr}
\toprule
{} &  count &   percent \\
classification                              &        &           \\
\midrule
Development Status :: 1 - Planning          &   2619 &  0.031979 \\
Development Status :: 2 - Pre-Alpha         &   6409 &  0.078256 \\
Development Status :: 3 - Alpha             &  22686 &  0.277003 \\
Development Status :: 4 - Beta              &  28960 &  0.353611 \\
Development Status :: 5 - Production/Stable &  20425 &  0.249396 \\
Development Status :: 6 - Mature            &    485 &  0.005922 \\
Development Status :: 7 - Inactive          &    314 &  0.003834 \\
\bottomrule
\end{tabular}



## Table 11: Intended Audience

In [91]:
audience_df = pd.read_sql_query("""
    SELECT pc.classification AS classification, COUNT(*) AS count
    FROM ossld_pypipackageclassification ppc
    JOIN ossld_pypiclassification pc ON ppc.pypi_classification_id = pc.id
    WHERE pc.classification LIKE 'Intend%'
    GROUP BY classification
    ORDER BY count DESC
    """, connection, index_col="classification")
audience_df = audience_df.sort_values("count", ascending=False)
audience_df.loc[:, "percent"] = audience_df["count"] / audience_df["count"].sum()
audience_df.index = [s.replace("Intended Audience :: ", "") for s in audience_df.index]
print(audience_df.to_latex())

\begin{tabular}{lrr}
\toprule
{} &  count &   percent \\
\midrule
Developers                       &  69682 &  0.668188 \\
Science/Research                 &  12020 &  0.115261 \\
System Administrators            &   7328 &  0.070269 \\
End Users/Desktop                &   4557 &  0.043698 \\
Information Technology           &   4132 &  0.039622 \\
Education                        &   2759 &  0.026456 \\
Financial and Insurance Industry &    872 &  0.008362 \\
Other Audience                   &    761 &  0.007297 \\
Telecommunications Industry      &    529 &  0.005073 \\
Healthcare Industry              &    526 &  0.005044 \\
Legal Industry                   &    426 &  0.004085 \\
Manufacturing                    &    383 &  0.003673 \\
Customer Service                 &    222 &  0.002129 \\
Religion                         &     88 &  0.000844 \\
\bottomrule
\end{tabular}



## Table 12: Operating System

In [92]:
os_df = pd.read_sql_query("""
    SELECT pc.classification AS classification, COUNT(*) AS count
    FROM ossld_pypipackageclassification ppc
    JOIN ossld_pypiclassification pc ON ppc.pypi_classification_id = pc.id
    WHERE pc.classification LIKE 'Operat%'
    GROUP BY classification
    ORDER BY count DESC
    """, connection, index_col="classification")
os_df = os_df.sort_values("count", ascending=False)
os_df.loc[:, "percent"] = os_df["count"] / os_df["count"].sum()
os_df.index = [s.replace("Operating System :: ", "") for s in os_df.index]
print(os_df.to_latex())

\begin{tabular}{lrr}
\toprule
{} &  count &   percent \\
\midrule
OS Independent                                 &  47053 &  0.577090 \\
POSIX                                          &   7850 &  0.096278 \\
POSIX :: Linux                                 &   7437 &  0.091212 \\
Microsoft :: Windows                           &   5258 &  0.064488 \\
MacOS :: MacOS X                               &   4631 &  0.056798 \\
Unix                                           &   4573 &  0.056086 \\
MacOS                                          &   2594 &  0.031815 \\
Microsoft                                      &    420 &  0.005151 \\
POSIX :: BSD                                   &    279 &  0.003422 \\
Microsoft :: Windows :: Windows 7              &    174 &  0.002134 \\
Microsoft :: Windows :: Windows 10             &    164 &  0.002011 \\
POSIX :: BSD :: FreeBSD                        &    138 &  0.001693 \\
Microsoft :: Windows :: Windows NT/2000        &    110 &  0.001349 \\
POSIX :: Su

In [93]:
os_df.loc[:, "classification_l1"] = [x.split("::")[0].strip() for x in os_df.index]
os_l1_df = os_df.groupby("classification_l1")["count", "percent"].sum()
os_l1_df = os_l1_df.sort_values("count", ascending=False)
print(os_l1_df.to_latex())

\begin{tabular}{lrr}
\toprule
{} &  count &   percent \\
classification\_l1 &        &           \\
\midrule
OS Independent    &  47053 &  0.577090 \\
POSIX             &  16036 &  0.196676 \\
MacOS             &   7240 &  0.088796 \\
Microsoft         &   6410 &  0.078617 \\
Unix              &   4573 &  0.056086 \\
Android           &     93 &  0.001141 \\
Other OS          &     49 &  0.000601 \\
iOS               &     44 &  0.000540 \\
OS/2              &     17 &  0.000208 \\
BeOS              &     11 &  0.000135 \\
PDA Systems       &      5 &  0.000061 \\
PalmOS            &      4 &  0.000049 \\
\bottomrule
\end{tabular}



## Table 13: Framework

In [94]:
framework_df = pd.read_sql_query("""
    SELECT pc.classification AS classification, COUNT(*) AS count
    FROM ossld_pypipackageclassification ppc
    JOIN ossld_pypiclassification pc ON ppc.pypi_classification_id = pc.id
    WHERE pc.classification LIKE 'Fram%'
    GROUP BY classification
    ORDER BY count DESC
    """, connection, index_col="classification")
framework_df = framework_df.sort_values("count", ascending=False)
framework_df.loc[:, "percent"] = framework_df["count"] / framework_df["count"].sum()
framework_df.index = [s.replace("Framework :: ", "") for s in framework_df.index]
print(framework_df.to_latex())

\begin{tabular}{lrr}
\toprule
{} &  count &   percent \\
\midrule
Django                     &  10815 &  0.294566 \\
Odoo                       &   5625 &  0.153207 \\
Plone                      &   3100 &  0.084434 \\
Django :: 1.11             &   1296 &  0.035299 \\
Django :: 1.8              &   1095 &  0.029824 \\
Django :: 2.0              &   1077 &  0.029334 \\
Django :: 1.9              &   1012 &  0.027564 \\
Django :: 1.10             &    948 &  0.025821 \\
Zope3                      &    942 &  0.025657 \\
Zope2                      &    931 &  0.025357 \\
Plone :: 4.3               &    735 &  0.020019 \\
Flask                      &    662 &  0.018031 \\
Tryton                     &    556 &  0.015144 \\
Pyramid                    &    527 &  0.014354 \\
Plone :: 4.2               &    493 &  0.013428 \\
Django :: 2.1              &    474 &  0.012910 \\
Plone :: 4.1               &    440 &  0.011984 \\
Buildout                   &    435 &  0.011848 \\
Pytest          

In [95]:
framework_df.loc[:, "classification_l1"] = [x.split("::")[0].strip() for x in framework_df.index]
framework_l1_df = framework_df.groupby("classification_l1")["count", "percent"].sum()
framework_l1_df = framework_l1_df.sort_values("count", ascending=False)
print(framework_l1_df.head(20).to_latex())

\begin{tabular}{lrr}
\toprule
{} &  count &   percent \\
classification\_l1 &        &           \\
\midrule
Django            &  17337 &  0.472205 \\
Plone             &   5983 &  0.162958 \\
Odoo              &   5625 &  0.153207 \\
Zope3             &    942 &  0.025657 \\
Zope2             &    931 &  0.025357 \\
Flask             &    662 &  0.018031 \\
Tryton            &    556 &  0.015144 \\
Pyramid           &    527 &  0.014354 \\
Buildout          &    502 &  0.013673 \\
Pytest            &    368 &  0.010023 \\
IPython           &    330 &  0.008988 \\
Twisted           &    324 &  0.008825 \\
AsyncIO           &    321 &  0.008743 \\
Sphinx            &    312 &  0.008498 \\
Pylons            &    267 &  0.007272 \\
TurboGears        &    187 &  0.005093 \\
Jupyter           &    168 &  0.004576 \\
Paste             &    151 &  0.004113 \\
Bob               &    139 &  0.003786 \\
Trac              &    107 &  0.002914 \\
\bottomrule
\end{tabular}



## Table 14: Topic

In [96]:
topic_df = pd.read_sql_query("""
    SELECT pc.classification AS classification, COUNT(*) AS count
    FROM ossld_pypipackageclassification ppc
    JOIN ossld_pypiclassification pc ON ppc.pypi_classification_id = pc.id
    WHERE pc.classification LIKE 'Topic%'
    GROUP BY classification
    ORDER BY count DESC
    """, connection, index_col="classification")
topic_df = topic_df.sort_values("count", ascending=False)
topic_df.loc[:, "percent"] = topic_df["count"] / topic_df["count"].sum()
topic_df.index = [s.replace("Topic :: ", "") for s in topic_df.index]
print(topic_df.to_latex())

\begin{tabular}{lrr}
\toprule
{} &  count &   percent \\
\midrule
Software Development :: Libraries :: Python Mod... &  22700 &  0.175384 \\
Utilities                                          &  12106 &  0.093533 \\
Internet :: WWW/HTTP                               &   7198 &  0.055613 \\
Software Development :: Libraries                  &   6781 &  0.052391 \\
Internet :: WWW/HTTP :: Dynamic Content            &   5736 &  0.044317 \\
Software Development                               &   5627 &  0.043475 \\
Scientific/Engineering                             &   4827 &  0.037294 \\
Software Development :: Build Tools                &   4056 &  0.031337 \\
Internet                                           &   2607 &  0.020142 \\
Software Development :: Testing                    &   2489 &  0.019230 \\
Scientific/Engineering :: Bio-Informatics          &   2050 &  0.015839 \\
Software Development :: Libraries :: Applicatio... &   1792 &  0.013845 \\
Database                          

In [97]:
topic_df.loc[:, "classification_l1"] = [x.split("::")[0].strip() for x in topic_df.index]
topic_l1_df = topic_df.groupby("classification_l1")["count", "percent"].sum()
topic_l1_df = topic_l1_df.sort_values("count", ascending=False)
print(topic_l1_df.to_latex())

\begin{tabular}{lrr}
\toprule
{} &  count &   percent \\
classification\_l1      &        &           \\
\midrule
Software Development   &  48887 &  0.377710 \\
Internet               &  19883 &  0.153620 \\
Scientific/Engineering &  16881 &  0.130426 \\
Utilities              &  12106 &  0.093533 \\
System                 &   9319 &  0.072000 \\
Text Processing        &   4445 &  0.034343 \\
Multimedia             &   3422 &  0.026439 \\
Database               &   2615 &  0.020204 \\
Communications         &   2518 &  0.019455 \\
Office/Business        &   2110 &  0.016302 \\
Security               &   1941 &  0.014997 \\
Documentation          &   1062 &  0.008205 \\
Games/Entertainment    &    951 &  0.007348 \\
Education              &    932 &  0.007201 \\
Terminals              &    583 &  0.004504 \\
Home Automation        &    421 &  0.003253 \\
Desktop Environment    &    345 &  0.002666 \\
Text Editors           &    290 &  0.002241 \\
Other/Nonlisted Topic  &    227 &  0.001

## Table 15: Subtopic - Software Development

In [98]:
topic_df = pd.read_sql_query("""
    SELECT pc.classification AS classification, COUNT(*) AS count
    FROM ossld_pypipackageclassification ppc
    JOIN ossld_pypiclassification pc ON ppc.pypi_classification_id = pc.id
    WHERE pc.classification LIKE 'Topic :: Software Development ::%'
    GROUP BY classification
    ORDER BY count DESC
    """, connection, index_col="classification")
topic_df = topic_df.sort_values("count", ascending=False)
topic_df.loc[:, "percent"] = topic_df["count"] / topic_df["count"].sum()
topic_df.index = [s.replace("Topic :: Software Development :: ", "") for s in topic_df.index]
print(topic_df.head(20).to_latex())

\begin{tabular}{lrr}
\toprule
{} &  count &   percent \\
\midrule
Libraries :: Python Modules         &  22700 &  0.524734 \\
Libraries                           &   6781 &  0.156750 \\
Build Tools                         &   4056 &  0.093759 \\
Testing                             &   2489 &  0.057536 \\
Libraries :: Application Frameworks &   1792 &  0.041424 \\
Quality Assurance                   &    969 &  0.022399 \\
User Interfaces                     &    681 &  0.015742 \\
Documentation                       &    543 &  0.012552 \\
Code Generators                     &    539 &  0.012460 \\
Version Control                     &    434 &  0.010032 \\
Embedded Systems                    &    264 &  0.006103 \\
Debuggers                           &    248 &  0.005733 \\
Widget Sets                         &    220 &  0.005086 \\
Compilers                           &    215 &  0.004970 \\
Interpreters                        &    170 &  0.003930 \\
Bug Tracking                      

## Table 16: Subtopic - Scientific/Engineering

In [99]:
topic_df = pd.read_sql_query("""
    SELECT pc.classification AS classification, COUNT(*) AS count
    FROM ossld_pypipackageclassification ppc
    JOIN ossld_pypiclassification pc ON ppc.pypi_classification_id = pc.id
    WHERE pc.classification LIKE 'Topic :: Scientific/Engineering ::%'
    GROUP BY classification
    ORDER BY count DESC
    """, connection, index_col="classification")
topic_df = topic_df.sort_values("count", ascending=False)
topic_df.loc[:, "percent"] = topic_df["count"] / topic_df["count"].sum()
topic_df.index = [s.replace("Topic :: Scientific/Engineering :: ", "") for s in topic_df.index]
print(topic_df.head(20).to_latex())

\begin{tabular}{lrr}
\toprule
{} &  count &   percent \\
\midrule
Bio-Informatics                      &   2050 &  0.170068 \\
Artificial Intelligence              &   1614 &  0.133897 \\
Mathematics                          &   1582 &  0.131243 \\
Information Analysis                 &   1574 &  0.130579 \\
Physics                              &   1253 &  0.103949 \\
Visualization                        &    979 &  0.081218 \\
Astronomy                            &    624 &  0.051767 \\
GIS                                  &    578 &  0.047951 \\
Chemistry                            &    444 &  0.036834 \\
Medical Science Apps.                &    415 &  0.034428 \\
Image Recognition                    &    239 &  0.019827 \\
Atmospheric Science                  &    236 &  0.019579 \\
Human Machine Interfaces             &    187 &  0.015514 \\
Interface Engine/Protocol Translator &    146 &  0.012112 \\
Electronic Design Automation (EDA)   &     92 &  0.007632 \\
Artificial Life    

## Number of import statements per year

In [53]:
import_year_frequency_list = []

for start_year in range(2005, 2020):
    print(start_year)
    query_df = pd.read_sql_query("""
        SELECT 
        --EXTRACT(YEAR FROM ppr.upload_time) AS release_year, ppri.import_name
        COUNT(*)
        FROM ossld_pypipackagereleaseimport ppri
        JOIN ossld_pypipackagerelease ppr ON ppri.pypi_package_release_id = ppr.id
        WHERE ppr.upload_time BETWEEN '{0}-01-01' AND '{1}-01-01'
        """.format(start_year, start_year+1), connection)
    import_year_frequency_list.append((start_year, query_df.iloc[0, 0]))
    
import_year_frequency_df = pd.DataFrame(import_year_frequency_list, columns=["year", "count"])
import_year_frequency_df.loc[:, "count"] = import_year_frequency_df["count"].astype(int)
import_year_frequency_df.set_index("year", drop=True, inplace=True)
print(import_year_frequency_df.to_latex())

2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
\begin{tabular}{lr}
\toprule
{} &     count \\
year &           \\
\midrule
2005 &     26756 \\
2006 &     91896 \\
2007 &    188050 \\
2008 &    459461 \\
2009 &   1015681 \\
2010 &   1520202 \\
2011 &   2335051 \\
2012 &   3552860 \\
2013 &   6837532 \\
2014 &  10864275 \\
2015 &  15716157 \\
2016 &  25696479 \\
2017 &  36932209 \\
2018 &  47745271 \\
2019 &   3834870 \\
\bottomrule
\end{tabular}



In [54]:
# calculate CAGR for new import statements
start_year = 2006
end_year = 2018

new_import_cagr = np.exp((np.log(import_year_frequency_df.loc[end_year, "count"]) - np.log(import_year_frequency_df.loc[start_year, "count"])) / (end_year - start_year + 1)) - 1
print("New import CAGR:", new_import_cagr)

New import CAGR: 0.6176885225732587
